## マルチエージェント: 出張手配 (ホテル/フライト) サンプル 日本語版
<!--
TRANSLATION_METADATA:
{
  "source_file": "08-multi-agent/code_samples/08-python_multi_aiagent_bookinghotel.ipynb",
  "language": "ja",
  "translated_at": "2025-08-28T00:00:00Z",
  "translator": "human-reviewed",
  "notes": "English explanatory markdown condensed for brevity."
}
-->
英語版 `08-python_multi_aiagent_bookinghotel.ipynb` を教育目的で要約翻訳。詳細英語コメントは日本語で凝縮し冗長さを削減。

目的: 予約エージェントと保存エージェントを SerpAPI / Azure AI Agent Service と連携させ、検索→Markdown整形→ファイル保存までの協調作業の例を示す。
> SerpAPI は .env ファイル上で API キーやエンドポイントを指定していないため、モックとして固定されたサンプルデータを本Notebookでは利用している。

## 全体フロー
1. 環境 & ライブラリ読み込み
2. CodeInterpreter 利用可否チェック
3. 設定 (エンドポイント/モデル) & 認証ヘルパ
4. 予約プラグイン (ホテル/フライト)
5. 保存プラグイン (旅程を Markdown 保存)
6. マルチエージェント実行 (ガード付き)

## .env 例
推奨 (どちらか一方で可):
```env
MODEL_DEPLOYMENT_NAME=gpt-4o-mini
AZURE_AI_AGENT_ENDPOINT=https://<resource>.services.ai.azure.com/api/projects/<project>
# もしくは (旧) connection string:
# AZURE_AI_AGENT_PROJECT_CONNECTION_STRING=endpoint=...;project=...;api-key=...
PROJECT_ENDPOINT=...  # AZURE_AI_AGENT_ENDPOINT 未指定時に補完
AZURE_AI_PROJECT_NAME=...          # connection string 動的組立用
SERPAPI_SEARCH_API_KEY=...         # 任意
SERPAPI_SEARCH_ENDPOINT=https://serpapi.com/search  # 任意
```
注意: api-key を使わない場合は Microsoft Entra (az login) 認証が必要。

## 依存ライブラリ (必要なら実行)
```bash
!pip install --upgrade azure-ai-agents semantic-kernel azure-identity requests python-dotenv
# CLI 利用が不安定な場合の更新
!pip install --upgrade azure-cli
```

In [1]:
# ===== 1. 基本インポート / 環境読み込み =====
# 学習ポイント:
# - .env には PROJECT_ENDPOINT / MODEL_DEPLOYMENT_NAME などを定義。api-key を含めない構成では AAD 認証(az login) を利用。
# - semantic-kernel の AzureAIAgentSettings は GA エンドポイント優先 (endpoint 引数) → 失敗時 legacy connection string。
import sys, pkgutil, os, asyncio, requests, hashlib, time, datetime, json, contextlib
from typing import Annotated
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential
from semantic_kernel.agents import AgentGroupChat, AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
print('# Python 実行ファイル:', sys.executable)
print('# azure-ai-agents インストール?:', pkgutil.find_loader('azure.ai.agents') is not None)
load_dotenv()  # .env をロード (必須環境変数: PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME など)

# Python 実行ファイル: c:\VSCode - Work\ai-agents-for-beginners\ai-agents-for-beginners\myenv\Scripts\python.exe
# azure-ai-agents インストール?: True


C:\Users\Mamoru Kuroda\AppData\Local\Temp\ipykernel_63384\2890377945.py:14: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  print('# azure-ai-agents インストール?:', pkgutil.find_loader('azure.ai.agents') is not None)


True

## Code Interpreter 利用可否確認

In [2]:
# ===== 2. CodeInterpreterTool 検出 & 再読み込みユーティリティ =====
# 学習ポイント:
# - CodeInterpreterTool が無くてもローカル保存で動作可能 (機能段階的フォールバック設計)。
# - refresh_code_interpreter() を設けることで pip アップグレード後にカーネル再起動せず再検出できる。
try:
    from azure.ai.agents.models import CodeInterpreterTool
    HAS_CODE_INTERPRETER = True
except Exception:
    CodeInterpreterTool = None  # type: ignore
    HAS_CODE_INTERPRETER = False
    print('[INFO] CodeInterpreterTool 未利用 → ローカル保存フォールバック')

from importlib import metadata as _md, import_module

def env_diagnostics():
    pkgs = ['azure-ai-agents','semantic-kernel','azure-identity']
    versions = {}
    for p in pkgs:
        try:
            versions[p] = _md.version(p)
        except Exception:
            versions[p] = 'not-installed'
    print('# パッケージバージョン:', versions)
    print('# CodeInterpreter 利用可否:', HAS_CODE_INTERPRETER)
    return versions

def refresh_code_interpreter(verbose: bool = True):
    global HAS_CODE_INTERPRETER, CodeInterpreterTool
    try:
        module = import_module('azure.ai.agents.models')
        CodeInterpreterTool = getattr(module, 'CodeInterpreterTool')  # type: ignore
        HAS_CODE_INTERPRETER = True
        if verbose:
            print('[REFRESH] CodeInterpreterTool を検出しました。')
    except Exception as e:
        HAS_CODE_INTERPRETER = False
        CodeInterpreterTool = None  # type: ignore
        if verbose:
            print('[REFRESH] 未検出:', e)
    return HAS_CODE_INTERPRETER

# 初期状態出力
env_diagnostics()

# パッケージバージョン: {'azure-ai-agents': '1.2.0b3', 'semantic-kernel': '1.36.2', 'azure-identity': '1.24.0'}
# CodeInterpreter 利用可否: True


{'azure-ai-agents': '1.2.0b3',
 'semantic-kernel': '1.36.2',
 'azure-identity': '1.24.0'}

## 再検出 (必要に応じ実行)

In [3]:
# CodeInterpreterTool 再検出トリガ
refresh_code_interpreter(); env_diagnostics()

[REFRESH] CodeInterpreterTool を検出しました。
# パッケージバージョン: {'azure-ai-agents': '1.2.0b3', 'semantic-kernel': '1.36.2', 'azure-identity': '1.24.0'}
# CodeInterpreter 利用可否: True


{'azure-ai-agents': '1.2.0b3',
 'semantic-kernel': '1.36.2',
 'azure-identity': '1.24.0'}

## 終了条件戦略 / 設定

In [4]:
# ===== 3. 設定 (GA エンドポイント / 旧 connection string) & 終了条件 =====
# 学習ポイント (シンプル版):
# 1. エンドポイントは AZURE_AI_AGENT_ENDPOINT が無ければ PROJECT_ENDPOINT から補完。
# 2. モデル名は MODEL_DEPLOYMENT_NAME のみ参照。
# 3. GA エンドポイント初期化が TypeError の場合は legacy connection string へ後方互換フォールバック。
# 4. 'saved' を含む発話でチャット終了 (ApprovalTerminationStrategy)。
class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return bool(history) and 'saved' in (history[-1].content or '').lower()

# (A) connection string 後方互換組立
raw_conn = os.getenv('AZURE_AI_AGENT_PROJECT_CONNECTION_STRING')
if not raw_conn:
    pe = os.getenv('PROJECT_ENDPOINT'); pn = os.getenv('AZURE_AI_PROJECT_NAME')
    if pe and pn:
        raw_conn = f'endpoint={pe};project={pn}'

# (B) GA endpoint 優先補完
endpoint = os.getenv('AZURE_AI_AGENT_ENDPOINT')
if not endpoint:
    pe = os.getenv('PROJECT_ENDPOINT')
    if pe:
        endpoint = pe.strip().strip('"').rstrip('/')
        os.environ['AZURE_AI_AGENT_ENDPOINT'] = endpoint
        print('# 補完 endpoint ->', endpoint)

# (C) モデル名 (MODEL_DEPLOYMENT_NAME 参照)
model_name = os.getenv('MODEL_DEPLOYMENT_NAME')
if not (endpoint and model_name):
    raise RuntimeError('endpoint / model_name が不足 (.env を再確認)')

# (D) Settings 初期化
try:
    ai_agent_settings = AzureAIAgentSettings(model_deployment_name=model_name, endpoint=endpoint)
    SETTINGS_MODE='GA-endpoint'
except TypeError:
    if not raw_conn:
        raise RuntimeError('endpoint 型初期化失敗 & connection string なし')
    ai_agent_settings = AzureAIAgentSettings(model_deployment_name=model_name, project_connection_string=raw_conn)
    SETTINGS_MODE='LEGACY-connection-string'

print('# Settings モード:', SETTINGS_MODE, '| model=', ai_agent_settings.model_deployment_name)

# (E) クライアント生成ヘルパ
from contextlib import asynccontextmanager
@asynccontextmanager
async def get_agents_client(credential):
    try:
        async with AzureAIAgent.create_client(credential=credential, endpoint=endpoint) as c:
            yield c
            return
    except TypeError:
        pass
    if not raw_conn:
        raise RuntimeError('クライアント生成失敗: fallback 不可')
    async with AzureAIAgent.create_client(credential=credential, conn_str=raw_conn) as c:
        yield c

# 補完 endpoint -> https://makuroda-foundry-learning.services.ai.azure.com/api/projects/makuroda-Foundry-Learning-project
# Settings モード: GA-endpoint | model= gpt-4o-mini


## 認証 / プラグイン / 実行 (以降のセル)
以下のセルでは:
1. 認証: DefaultAzureCredential のみ (VS Code / Azure Sign-In / Managed Identity 等に委ねる)
2. 予約プラグイン: SerpAPI (キー無ならモック) + キャッシュ
3. 保存プラグイン: CodeInterpreterTool 有無で保存方式フォールバック
4. 実行セル: AgentGroupChat + ガード (重複 / 時間 / 'saved')
---

In [5]:
# 認証ヘルパ (学習用にシンプル化: DefaultAzureCredential のみ)
# - VS Code で Azure にサインイン済み / az login 済み / Managed Identity などに自動対応
# - CLI タイムアウト雑音を避けるため AzureCliCredential フォールバックを削除
async def get_credential():
    scope = 'https://cognitiveservices.azure.com/.default'
    dac = DefaultAzureCredential(exclude_cli_credential=True, exclude_visual_studio_code_credential=False)
    await dac.get_token(scope)  # 失敗時は例外で明確化
    print('[AUTH] DefaultAzureCredential')
    return dac

In [6]:
# 予約/保存プラグイン (SerpAPI 無→モック継続 / CodeInterpreter 有無で保存手段切替)
SERPAPI_SEARCH_API_KEY = os.getenv('SERPAPI_SEARCH_API_KEY')
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT') or 'https://serpapi.com/search'
if not os.getenv('SERPAPI_SEARCH_ENDPOINT'):
    print('[WARN] SERPAPI_SEARCH_ENDPOINT 未設定 → 既定 URL 使用')
if not SERPAPI_SEARCH_API_KEY:
    print('[WARN] SERPAPI_SEARCH_API_KEY 未設定 → モック応答 (決定論的サンプル)')

class BookingPlugin:
    def __init__(self):
        self._cache={}; self._timeout=15
    def _cache_key(self,*parts):
        return hashlib.sha256('|'.join(map(str,parts)).encode()).hexdigest()[:16]
    def _http(self, params):
        # モック: SerpAPI キー無しなら決定論的なサンプル構造を返す
        if not SERPAPI_SEARCH_API_KEY:
            if params.get('engine')=='google_hotels':
                return {
                    'status':'mock','properties':[{
                        'name':'BizStay Midtown','rate_per_night':'USD 210','overall_rating':4.3
                    },{
                        'name':'Riverside Comfort','rate_per_night':'USD 185','overall_rating':4.1
                    },{
                        'name':'Central Hub Hotel','rate_per_night':'USD 240','overall_rating':4.6
                    }]
                }
            if params.get('engine')=='google_flights':
                # 簡易化: 片道/復路両方同じ構造を返却 (呼び出し側でまとめる)
                return {
                    'status':'mock','flights':[{
                        'airline':'Contoso Air','duration':'7h45m','carrier':'CA','price':'USD 980'
                    }]
                }
            return {'status':'mock'}
        try:
            r = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params, timeout=self._timeout)
            if r.status_code!=200:
                return {'status':'http_error','code':r.status_code}
            return r.json()
        except Exception as e:
            return {'status':'exception','error':str(e)}
    def _hotels(self,data):
        props=(data.get('properties') if isinstance(data,dict) else []) or []
        lines=[f"- {p.get('name') or p.get('title') or 'N/A'} | 料金:{p.get('rate_per_night') or p.get('price') or '—'} | 評価:{p.get('overall_rating') or p.get('rating') or '—'}" for p in props[:3]]
        return '\n'.join(lines) if lines else '- (結果なし)'
    def _flights(self,data):
        if not isinstance(data,dict): return '- (結果なし)'
        def pick(seg):
            if not isinstance(seg,dict): return 'N/A'
            arr=seg.get('best_flights') or seg.get('flights') or []
            if arr:
                leg=arr[0]; return f"{leg.get('airline') or leg.get('carrier') or '航空会社'},{leg.get('duration') or '—'},{leg.get('price') or seg.get('price') or '—'}"
            return '—'
        outbound=pick(data.get('outbound'))
        inbound=pick(data.get('return')) if data.get('return') else '—'
        return f"- 往路:{outbound}\n- 復路:{inbound}"
    @kernel_function(description='ホテル検索')
    def booking_hotel(self, query: Annotated[str,'都市'], check_in_date: Annotated[str,'IN'], check_out_date: Annotated[str,'OUT']) -> Annotated[str,'Hotel']:
        k=self._cache_key('h',query,check_in_date,check_out_date)
        if k in self._cache: return self._cache[k]
        params={'engine':'google_hotels','q':query,'check_in_date':check_in_date,'check_out_date':check_out_date,'adults':'2','currency':'USD','hl':'en','api_key':SERPAPI_SEARCH_API_KEY}
        data=self._http(params); res=f"## ホテル ({query} {check_in_date}→{check_out_date})\n"+self._hotels(data); self._cache[k]=res; return res
    @kernel_function(description='フライト検索')
    def booking_flight(self, origin: Annotated[str,'出発'], destination: Annotated[str,'到着'], outbound_date: Annotated[str,'往路日'], return_date: Annotated[str,'復路日']) -> Annotated[str,'Flight']:
        k=self._cache_key('f',origin,destination,outbound_date,return_date)
        if k in self._cache: return self._cache[k]
        def _q(dep,arr,out,ret): p={'engine':'google_flights','departure_id':dep,'arrival_id':arr,'outbound_date':out,'return_date':ret,'currency':'USD','hl':'en','api_key':SERPAPI_SEARCH_API_KEY}; return self._http(p)
        data={'outbound':_q(origin,destination,outbound_date,return_date),'return':_q(destination,origin,return_date,outbound_date)}
        res=f"## フライト ({origin}->{destination} {outbound_date}/{return_date})\n"+self._flights(data); self._cache[k]=res; return res

class SavePlugin:
    def __init__(self):
        self._saved=False
    @kernel_function(description='旅程を保存 (ローカル, 単回)')
    async def saving_plan(self, tripplan: Annotated[str,'旅程']) -> Annotated[str,'結果']:
        if self._saved:
            return 'already_saved'
        os.makedirs('trip', exist_ok=True)
        path='trip/trip-plan.md'
        with open(path,'w',encoding='utf-8') as f:
            f.write(tripplan)
        self._saved=True
        print('# ローカル保存完了:', path)
        return 'saved: file=' + path

[WARN] SERPAPI_SEARCH_ENDPOINT 未設定 → 既定 URL 使用
[WARN] SERPAPI_SEARCH_API_KEY 未設定 → モック応答 (決定論的サンプル)


In [7]:
# 実行ロジック (2エージェント + 'saved' 終了 + 重複/時間ガード)
async def run_chat():
    creds = await get_credential()
    async with (creds, get_agents_client(credential=creds) as client):
        booking_def = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name='BookingAgent',
            instructions='あなたは旅行予約アシスタント。フライトとホテル案を取得 (ツール) し、Markdown 旅程を統合して SaveAgent に一度だけ明確に引き渡してください。',
        )
        booking = AzureAIAgent(client=client, definition=booking_def)
        booking.kernel.add_plugin(BookingPlugin(), plugin_name='booking')

        save_def = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name='SaveAgent',
            instructions=(
                'あなたは旅程保存アシスタント。受け取った最終 Markdown 旅程をローカルに保存し、1回だけ "saved" を含む短い完了メッセージ (例: "saved: file=trip/trip-plan.md") を返して終了。'
                ' 以後そのような保存完了メッセージは繰り返さないでください。'
            ),
        )
        saver = AzureAIAgent(client=client, definition=save_def)
        saver.kernel.add_plugin(SavePlugin(), plugin_name='saving')

        chat = AgentGroupChat(
            agents=[booking, saver],
            termination_strategy=ApprovalTerminationStrategy(agents=[saver], maximum_iterations=6)
        )
        user_input = '2025年2月20日〜2月27日にロンドン発ニューヨーク出張。往復航空券とホテルを手配し最終 Markdown 旅程を保存してください。'
        await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

        start=time.time(); MAX_SECONDS=45; last=None; repeat=0; MAX_REPEAT=2
        saver_msgs=[]; early_terminated=False; saved_once=False
        agen = chat.invoke()
        try:
            async for content in agen:
                text = content.content or ''
                payload=f"{content.role}:{content.name}:{text}"
                if content.name == 'SaveAgent':
                    lowered=text.lower()
                    if 'saved:' in lowered:
                        if saved_once:
                            # 二重 saved は無視して終了
                            print('[GUARD] 二重 saved 無視して終了'); await agen.aclose(); break
                        saved_once=True
                        saver_msgs.append(text)
                        print('[EARLY] SaveAgent saved 検出 → 終了')
                        await agen.aclose(); early_terminated=True; break
                    elif 'already_saved' in lowered:
                        # 既に保存済みのシグナルは表示のみ
                        print('[INFO] already_saved シグナルを無視')
                    else:
                        saver_msgs.append(text)
                # 重複/時間ガード
                if payload == last:
                    repeat +=1
                    if repeat>=MAX_REPEAT:
                        print('[GUARD] 重複出力上限'); await agen.aclose(); break
                else:
                    repeat=0; last=payload
                if time.time()-start>MAX_SECONDS:
                    print('[GUARD] 時間上限'); await agen.aclose(); break
                agent_label = content.name if content.name else '*'
                print(f"# {content.role} - {agent_label}: {text}")
        finally:
            with contextlib.suppress(Exception):
                await agen.aclose()

        complete = chat.is_complete or early_terminated
        print('# 完了フラグ(表示用):', complete)

        if saver_msgs:
            print('# Saver 出力まとめ:')
            for m in saver_msgs[-5:]:
                print('#', m)

        with contextlib.suppress(Exception):
            await chat.reset()
        with contextlib.suppress(Exception):
            await client.agents.delete_agent(saver.id)
        with contextlib.suppress(Exception):
            await client.agents.delete_agent(booking.id)

# Notebook ループ判定 (既存イベントループ内ならそのまま await)
async def _launch():
    try:
        loop=asyncio.get_running_loop()
        if loop.is_running():
            print('# 既存ループ: await で実行')
            await run_chat(); return
    except RuntimeError:
        pass
    print('# 新規ループ実行')
    await run_chat()

# 実行は次セルで "await _launch()" を呼び出す (スクリプト実行時は: asyncio.run(run_chat()))

In [ ]:
# 実行トリガ (_launch() でチャット開始)
await _launch()

# 既存ループ: await で実行
[AUTH] DefaultAzureCredential
[AUTH] DefaultAzureCredential
# AuthorRole.ASSISTANT - BookingAgent: 以下は、2025年2月20日から2月27日までのロンドン発ニューヨーク出張の最終Markdown旅程です：

## フライト (ロンドン->ニューヨーク 2025-02-20/2025-02-27)
- **往路**: Contoso Air, 7時間45分, USD 980
- **復路**: Contoso Air, 7時間45分, USD 980

## ホテル (ニューヨーク 2025-02-20→2025-02-27)
- **BizStay Midtown** | 料金: USD 210 | 評価: 4.3
- **Riverside Comfort** | 料金: USD 185 | 評価: 4.1
- **Central Hub Hotel** | 料金: USD 240 | 評価: 4.6

この旅程を SaveAgent に保存します。
# AuthorRole.ASSISTANT - BookingAgent: 以下は、2025年2月20日から2月27日までのロンドン発ニューヨーク出張の最終Markdown旅程です：

## フライト (ロンドン->ニューヨーク 2025-02-20/2025-02-27)
- **往路**: Contoso Air, 7時間45分, USD 980
- **復路**: Contoso Air, 7時間45分, USD 980

## ホテル (ニューヨーク 2025-02-20→2025-02-27)
- **BizStay Midtown** | 料金: USD 210 | 評価: 4.3
- **Riverside Comfort** | 料金: USD 185 | 評価: 4.1
- **Central Hub Hotel** | 料金: USD 240 | 評価: 4.6

この旅程を SaveAgent に保存します。
# ローカル保存完了: trip/trip-plan.md
# ローカル保存完了: trip/trip-plan.md
[EARLY] SaveAge

: 